In [1]:
# Load model directly
from transformers import AutoTokenizer, AutoModelForSequenceClassification

# tokenizer = AutoTokenizer.from_pretrained("./fine-tuned-distilbert-base-uncased")
tokenizer = AutoTokenizer.from_pretrained('distilbert-base-uncased')
model = AutoModelForSequenceClassification.from_pretrained("./fine-tuned-distilbert-base-uncased")

/Users/nojeda/repo/financial-sentiment-analysis-distilbert-base-uncased/.venv/lib/python3.9/site-packages/urllib3/__init__.py:35: NotOpenSSLWarning: urllib3 v2 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(


In [2]:
import shap
import torch

# Define a prediction function
def f(x):
    # Tokenize input texts with attention mask and return as PyTorch tensor
    encoded = [tokenizer.encode_plus(
        v,
        add_special_tokens=True,
        max_length=512,
        padding='max_length',
        truncation=True,
        return_attention_mask=True,
        return_tensors='pt'
    ) for v in x]
    
    # Stack all tensors into batches
    input_ids = torch.cat([e['input_ids'] for e in encoded])
    attention_mask = torch.cat([e['attention_mask'] for e in encoded])
    
    # Get model outputs with proper attention masking
    outputs = model(input_ids, attention_mask=attention_mask)[0]
    
    # Convert to probabilities
    probs = torch.nn.functional.softmax(outputs, dim=1)
    return probs

# Create an explainer object with a larger masker output
explainer = shap.Explainer(f, tokenizer, output_names=['Negative', 'Neutral', 'Positive'])

# Create more diverse example sentences for better explanation
example_texts = [
    "I absolutely love this product, it's amazing!",
    "This is the worst experience ever, very disappointing",
    "The item was okay, nothing special"
]

# Generate and visualize SHAP values with more detailed plotting
shap_values = explainer(example_texts)
shap.plots.text(shap_values, display=True)  # Set display=False for better notebook integration

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


  0%|          | 0/156 [00:00<?, ?it/s]

PartitionExplainer explainer:  33%|███▎      | 1/3 [00:00<?, ?it/s]

  0%|          | 0/110 [00:00<?, ?it/s]

PartitionExplainer explainer: 100%|██████████| 3/3 [00:26<00:00,  4.30s/it]

  0%|          | 0/72 [00:00<?, ?it/s]

PartitionExplainer explainer: 4it [00:33, 11.18s/it]                       
